<a href="https://colab.research.google.com/github/niest-pc/Ejercicios-Ciencia_de_Datos/blob/main/05_analisisAgrupamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Realizando primer Análisis de Agrupamiento

Capítulo 5 del libro The Data Science Workshop. Second Edition. Packt Publishing, 2020.

El análisis de agrupamiendo con el algoritmo k-medias es muy popular en la industria para segmentar perfiles de clientes, así como para detectar transacciones sospechosas o anomalías.

En este capítulo:
- Se aprende sobre diversos conceptos, como los centroides y la distancia euclidiana al cuadrado.
- Se aprenden los principales hiperparámetros de k-medias: init (método de inicialización), n_init (número de ejecuciones de inicialización), n_clusters (número de grupos) y random_state (semilla especificada).
- Se analiza la importancia de elegir el número óptimo de conglomerados, inicializar correctamente los centroides y estandarizar los datos.

## Agrupamiento con k-medias

En este capítulo se trabaja con un conjunto de datos de código abierto compartido públicamente por la Oficina de Impuestos de Australia (ATO). El conjunto de datos contiene estadísticas sobre cada código postal en Australia durante el año fiscal 2014-15.

### Ejercicio 5.01: Realización de Análisis de Agrupamiento
En este ejercicio se utiliza la agrupación en clústeres de k-medias en el conjunto de datos ATO y se observan los diferentes clústeres en que se divide el conjunto de datos, analizando el resultado.

In [1]:
# Importando librerías
import pandas as pd
from sklearn.cluster import KMeans
import altair as alt
import numpy as np

In [2]:
# Variable con la url del archivo con los datos
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/'\
          'The-Data-Science-Workshop/master/Chapter05/DataSet/taxstats2015.csv'

# Cargando los datos. Con 'usecols' se pueden cargar solo las columnas necesarias
df = pd.read_csv(file_url, usecols=['Postcode', 'Average net tax',
                                    'Average total deductions'])

# Mostrando las primeras filas del dataframe
df.tail()

,Postcode,Average total deductions,Average net tax
2468,870,2377,14788
2469,872,1218,9017
2470,880,2309,16574
2471,885,3039,28795
2472,886,2191,18141


In [3]:
# Instanciando el algoritmo
kmeans = KMeans(random_state = 42)

# Seleccionando los datos de entrenamiento
X = df[['Average net tax', 'Average total deductions']]

# Ajustando el algoritmo con los datos de entrenamiento
kmeans.fit(X)

# Calculando la asignación de agrupamiento a partir de la variable de entrada
df['Cluster'] = kmeans.predict(X)

# Mostrando las primeras filas del dataframe
df.head()

,Postcode,Average total deductions,Average net tax,Cluster
0,2000,2071,27555,7
1,2006,3804,28142,7
2,2007,1740,15649,6
3,2008,3917,53976,5
4,2009,3433,32430,1


In [4]:
# Instanciando un gráfico de dispersión
scatter_plot1 = alt.Chart(df).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot1.encode(x = 'Average net tax',
                     y = 'Average total deductions',
                     color = 'Cluster:N',
                     tooltip = ['Postcode', 'Cluster',
                                'Average total deductions',
                                'Average net tax']).properties(width = 495,
                                                               height = 330).interactive()

alt.Chart(...)

### Ejercicio 5.02: Agrupación de códigos postales australianos por ingresos y gastos empresariales
En este ejercicio se realiza un análisis de agrupamiento con k-medias y se visualizan los resultados en función de los valores de los códigos postales ordenados por ingresos y gastos empresariales.

In [5]:
# Cargando los datos
df2 = pd.read_csv(file_url, usecols=['Postcode', 'Average total business income',
                                    'Average total business expenses'])

# Inspeccionando el dataframe
df2.tail()

,Postcode,Average total business income,Average total business expenses
2468,870,62793,44687
2469,872,53025,45670
2470,880,45603,28700
2471,885,53148,39850
2472,886,121057,90120


In [6]:
# Extrayendo los datos de entrenamiento
X2 = df2[['Average total business income', 'Average total business expenses']]

# Instanciando el modelo
kmeans2 = KMeans(random_state = 8)

# Ajustando el modelo con los datos de entrenamiento
kmeans2.fit(X2)

# Prediciendo la asignación de agrupamiento a partir de la variable de entrada
df2['Cluster'] = kmeans2.predict(X2)

# Mostrando las primeras filas del dataframe
df2.head(10)

,Postcode,Average total business income,Average total business expenses,Cluster
0,2000,210901,222191,3
1,2006,69983,48971,5
2,2007,575099,639499,2
3,2008,53329,32173,1
4,2009,237539,222993,3
5,2010,90741,56258,5
6,2011,113791,74447,0
7,2015,64191,43379,1
8,2016,75671,48255,5
9,2017,113216,97041,0


In [7]:
# Generando una tabla dinámica con los promedios de las dos columnas para cada valor del clúster
df2.pivot_table(values = ['Average total business income',
                         'Average total business expenses'], # columnas a agregar
               index = 'Cluster',                            # columna a agrupar
               aggfunc = "mean")                            # función de agregación

,Average total business expenses,Average total business income
Cluster,,
0,86049.286550,108267.717349
1,38775.765319,54345.993481
2,672866.000000,671307.000000
3,201228.777778,246987.682540
4,400409.000000,447396.111111
5,59881.993534,78207.634698
6,129461.486772,157741.835979
7,865605.500000,873123.000000


In [8]:
# Instanciando un gráfico de dispersión
scatter_plot = alt.Chart(df2).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot.encode(x = 'Average total business income',   # Etiqueta en el eje x
                    y = 'Average total business expenses', # Etiqueta en el eje y
                    color = 'Cluster:N',                   # Coloreando cada grupo
                    tooltip = ['Postcode', 'Cluster',      # Información mostrada
                               'Average total business income',
                               'Average total business expenses']).properties(width = 495,
                                                                               height = 330).interactive()

alt.Chart(...)

## Elección del número de clústeres

### Ejercicio 5.03: Determinación del número óptimo de clústeres
En este ejercicio se aplica el método Elbow para determinar el número óptimo de clústeres, antes de ajustar un modelo de k-medias.

Se utilizan los mismos datos cargados en el Ejercicio 5.02.

In [9]:
# Cargando los datos
df3 = pd.read_csv(file_url, usecols=['Postcode', 'Average total business income',
                                    'Average total business expenses'])

# Extrayendo los datos de entrenamiento
X3 = df3[['Average total business income', 'Average total business expenses']]

# Creando un dataframe
clusters = pd.DataFrame()

# Creando una lista
inertia = []

# Asignando una nueva columna con el rango de números de clúster, del 1 al 14
clusters['cluster_range'] = range(1, 15)

# Recorriendo cada clúster, ajustandoles un modelo k-medias y
# agregando los valores a la lista 'inertia'
for k in clusters['cluster_range']:
  kmeans = KMeans(n_clusters=k).fit(X3)
  inertia.append(kmeans.inertia_)

# Agregando la lista 'inertia' al dataframe 'clusters'
clusters['inertia'] = inertia
clusters

,cluster_range,inertia
0,1,1.333516e+13
1,2,7.063394e+12
2,3,3.719188e+12
3,4,2.371989e+12
4,5,1.713743e+12
5,6,1.256158e+12
6,7,9.625622e+11
7,8,8.862210e+11
8,9,7.036027e+11
9,10,5.637758e+11


In [10]:
# Trazando el gráfico Elbow
alt.Chart(clusters).mark_line().encode(x = 'cluster_range',
                                       y = 'inertia').properties(width = 495,
                                                                 height = 330)

alt.Chart(...)

In [11]:
# Identificando el número óptimo de clústeres y asignando el valor a una variable
clusters_opt = 4

# Entrenando un modelo k-means con esta cantidad de clústeres
kmeans = KMeans(random_state = 42, n_clusters = clusters_opt)

# Ajustando el modelo con los datos de entrenamiento
kmeans.fit(X3)

# Asignación de agrupamiento a partir de la variable de entrada
df3['Cluster3'] = kmeans.predict(X3)

df3.head()

,Postcode,Average total business income,Average total business expenses,Cluster3
0,2000,210901,222191,1
1,2006,69983,48971,0
2,2007,575099,639499,3
3,2008,53329,32173,0
4,2009,237539,222993,1


In [12]:
# Instanciando un gráfico de dispersión
scatter_plot3 = alt.Chart(df3).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot3.encode(x = 'Average total business income',
                     y = 'Average total business expenses',
                     color = 'Cluster3:N',
                     tooltip = ['Postcode', 'Cluster3',
                                'Average total business income',
                                'Average total business expenses']).properties(width = 495,
                                                                               height = 330).interactive()

alt.Chart(...)

## Inicialización de Clústeres

### Ejercicio 5.04: Uso de diferentes parámetros de inicialización para lograr un resultado adecuado
En este ejercicio se prueban diferentes valores para los hiperparámetros *init* y _n_init_ para observar cómo afectan el resultado final de la agrupación.

Se utilizan los mismos datos que en el Ejercicio 5.02.

In [13]:
# Cargando los datos
df4 = pd.read_csv(file_url, usecols=['Postcode', 'Average total business income',
                                    'Average total business expenses'])

# Extrayendo los datos de entrenamiento
X4 = df4[['Average total business income', 'Average total business expenses']]

# Ajustando un modelo k-medias con "n_init = 1" e "init = 'random'"
kmeans4 = KMeans(random_state = 42, n_clusters = clusters_opt,
                 n_init = 1, init = 'random')

# Ajustando el modelo con los datos de entrenamiento
kmeans4.fit(X4)

# Asignación de agrupamiento a partir de la variable de entrada
df4['Cluster4'] = kmeans4.predict(X4)

df4.head()

,Postcode,Average total business income,Average total business expenses,Cluster4
0,2000,210901,222191,3
1,2006,69983,48971,1
2,2007,575099,639499,2
3,2008,53329,32173,1
4,2009,237539,222993,3


In [14]:
# Instanciando un gráfico de dispersión
scatter_plot4 = alt.Chart(df4).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot4.encode(x = 'Average total business income',
                     y = 'Average total business expenses',
                     color = 'Cluster4:N',
                     tooltip = ['Average total business income',
                                'Average total business expenses']).properties(width = 495,
                                                                               height = 330).interactive()

alt.Chart(...)

In [15]:
# Probando con diferentes hiperparámetros: "n_init = 10" e "init = 'random'"
kmeans42 = KMeans(random_state = 42, n_clusters = clusters_opt,
                 n_init = 10, init = 'random')

# Ajustando el modelo con los datos de entrenamiento
kmeans42.fit(X4)

# Asignación de grupo a partir de la variable de entrada
df4['Cluster4'] = kmeans42.predict(X4)

df4.head()

,Postcode,Average total business income,Average total business expenses,Cluster4
0,2000,210901,222191,2
1,2006,69983,48971,0
2,2007,575099,639499,1
3,2008,53329,32173,0
4,2009,237539,222993,2


In [16]:
# Instanciando un gráfico de dispersión
scatter_plot42 = alt.Chart(df4).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot4.encode(x = 'Average total business income',
                     y = 'Average total business expenses',
                     color = 'Cluster4:N',
                     tooltip = ['Average total business income',
                                'Average total business expenses']).properties(width = 495,
                                                                               height = 330).interactive()

alt.Chart(...)

## Cálculo de la distancia al centroide

### Ejercicio 5.05: Encontrar los centroides más cercanos en nuestro conjunto de datos
En este ejercicio se codifica la primera iteración de k-medias para asignar los puntos de datos a sus centroides de clúster más cercanos.

Se utilizan los mismos datos que en el Ejercicio 5.02.

In [17]:
# Cargando datos
df5 = pd.read_csv(file_url, usecols=['Postcode', 'Average total business income',
                                    'Average total business expenses'])

# Extrayendo los datos de entrenamiento
X5 = df5[['Average total business income', 'Average total business expenses']]

# Calculando el mínimo y el máximo de las variables de entrenamiento
business_income_min = df5['Average total business income'].min()
business_income_max = df5['Average total business income'].max()
business_expenses_min = df5['Average total business expenses'].min()
business_expenses_max = df5['Average total business expenses'].max()

# Imprimiendo resultados
print('Business Income Min: ', business_income_min)
print('Business Income Max: ', business_income_max)
print('Business Expenses Min: ', business_expenses_min)
print('Business Expenses Max: ', business_expenses_max)

Business Income Min:  0
Business Income Max:  876324
Business Expenses Min:  0
Business Expenses Max:  884659


In [18]:
# Importando libreria random
import random

# Fijando semilla
random.seed(9)

# Creando dataframe
centroides = pd.DataFrame()

# Generando 4 valores aleatorios con valores entre los mínimos y máximos calculados para 'business income'
centroides['Average total business income'] = random.sample(range(business_income_min,
                                                                  business_income_max), 4)

# Generando 4 valores aleatorios con valores entre los mínimos y máximos calculados para 'business expenses'
centroides['Average total business expenses'] = random.sample(range(business_expenses_min,
                                                                  business_expenses_max), 4)

# Creando una nueva columna para asignarle un indice a cada cluster
centroides['Cluster'] = centroides.index
centroides

,Average total business income,Average total business expenses,Cluster
0,485498,145269,0
1,643002,195187,1
2,391445,709512,2
3,280110,6747,3


In [19]:
# Creando un gráfico de dispersión para mostrar los datos contenidos en df5
plot5 = alt.Chart(df5).mark_circle().encode(x = 'Average total business income',
                                                   y = 'Average total business expenses',
                                                   color = alt.value('orange'),
                                                   tooltip = ['Postcode',
                                                              'Average total business income',
                                                              'Average total business expenses']).properties(width = 495,
                                                                                                             height = 330).interactive()

# Creando un gráfico de dispersión para mostrar los centroides
plotc = alt.Chart(centroides).mark_circle().encode(x = 'Average total business income',
                                                   y = 'Average total business expenses',
                                                   color = alt.value('black'),
                                                   tooltip = ['Cluster',
                                                              'Average total business income',
                                                              'Average total business expenses']).properties(width = 495,
                                                                                                             height = 330).interactive()

# Mostrando ambos gráficos
plot5 + plotc

alt.LayerChart(...)

In [20]:
# Definiendo una función que calcula la distancia euclidiana al cuadrado y devuelve su valor.
# Esta función toma las coordenadas 'x' y 'y' de un punto de datos y un centroide
def squared_euclidean(data_x, data_y, centroide_x, centroide_y, ):
  return (data_x - centroide_x)**2 + (data_y - centroide_y)**2

In [21]:
# Extrayendo las coordenadas 'x' y 'y' de la primera fila y guardandolas en variables
data_x = df5.at[0, 'Average total business income']
data_y = df5.at[0, 'Average total business expenses']

# Usando un bucle for para calcular la distancia euclidiana al cuadrado de la primera observación
# contra los 4 centroides y guardando el resultado
distancias = [squared_euclidean(data_x,
                                data_y,
                                centroides.at[i, 'Average total business income'],
                                centroides.at[i, 'Average total business expenses']) for i in range(4)]

# Desplegando el resultado
distancias

[np.int64(81320506493),
 np.int64(187440490217),
 np.int64(270077892977),
 np.int64(51206002817)]

In [22]:
# Encontrando el indice del centroide más cercano
cluster_index = distancias.index(min(distancias))

# Guardando el indice en el dataframe
df5.at[0, 'Cluster'] = cluster_index

# Observando el resultado
df5.head()

,Postcode,Average total business income,Average total business expenses,Cluster
0,2000,210901,222191,3.0
1,2006,69983,48971,NaN
2,2007,575099,639499,NaN
3,2008,53329,32173,NaN
4,2009,237539,222993,NaN


In [23]:
# Haciendo el proceso anterior para todos los datos
for i in range(len(df5)):
  data_x = df5.at[i, 'Average total business income']
  data_y = df5.at[i, 'Average total business expenses']
  distancias = [squared_euclidean(data_x,
                                  data_y,
                                  centroides.at[j, 'Average total business income'],
                                  centroides.at[j, 'Average total business expenses']) for j in range(4)]
  cluster_index = distancias.index(min(distancias))
  df5.at[i, 'Cluster'] = cluster_index

# Mostrando las primeras filas del dataframe
df5.head()

,Postcode,Average total business income,Average total business expenses,Cluster
0,2000,210901,222191,3.0
1,2006,69983,48971,3.0
2,2007,575099,639499,2.0
3,2008,53329,32173,3.0
4,2009,237539,222993,3.0


In [24]:
# Creando un gráfico de dispersión para mostrar los datos contenidos en df5
plot52 = alt.Chart(df5).mark_circle().encode(x='Average total business income',
                                             y='Average total business expenses',
                                             color='Cluster:N',
                                             tooltip=['Postcode', 'Cluster',
                                                      'Average total business income',
                                                      'Average total business expenses']).interactive()

# Gráficando los datos junto con los centroides
plot52 + plotc

alt.LayerChart(...)

## Estandarización de datos

### Ejercicio 5.06: Estandarización de los datos del conjunto de datos
En este ejercicio se estandarizan los datos mediante el escalamiento mínimo-máximo y la puntuación z, ajustando un modelo de k-medias para cada método y analizando su impacto en las k-medias.

Se utilizan los mismos datos que en el Ejercicio 5.01.

In [25]:
# Cargando los datos
df6 = pd.read_csv(file_url, usecols=['Postcode', 'Average net tax',
                                    'Average total deductions'])

# Extrayendo los datos de entrenamiento
X6 = df6[['Average net tax', 'Average total deductions']]

In [26]:
# Importando las clases para realizar el escalamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [27]:
# Instanciando el escalador MinMaxScaler
min_max_scaler = MinMaxScaler()

# Ajustando los datos
min_max_scaler.fit(X6)

# Escalando los datos
X_min_max = min_max_scaler.transform(X6)

# Mostrando los datos escalados
X_min_max

array([[0.25619932, 0.05804452],
       [0.26313737, 0.1278026 ],
       [0.11547644, 0.04472085],
       ...,
       [0.12640947, 0.06762468],
       [0.27085549, 0.09700922],
       [0.14493062, 0.06287485]])

In [28]:
# Instanciando el algoritmo
kmeans6 = KMeans(random_state = 42, n_clusters = 4)

# Ajustando el algoritmo con los datos escalados
kmeans6.fit(X_min_max)

# Asignando grupos a partir de la variable de entrada
df6['cluster61'] = kmeans6.predict(X_min_max)

# Revisando los datos
df6.head()

,Postcode,Average total deductions,Average net tax,cluster61
0,2000,2071,27555,3
1,2006,3804,28142,1
2,2007,1740,15649,0
3,2008,3917,53976,1
4,2009,3433,32430,1


In [29]:
# Instanciando un gráfico de dispersión
scatter_plot61 = alt.Chart(df6).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot61.encode(x = 'Average net tax',
                      y = 'Average total deductions',
                      color = 'cluster61:N',
                      tooltip = ['Postcode', 'cluster61',
                                 'Average total deductions',
                                 'Average net tax']).properties(width = 495,
                                                                height = 330).interactive()

alt.Chart(...)

In [30]:
# Instanciando el escalador StandardScaler
standard_scaler = StandardScaler()

# Ajustando los datos
standard_scaler.fit(X6)

# Escalando los datos
X_standard = standard_scaler.transform(X6)

# Mostrando los datos escalados
X_standard

array([[ 1.47818338, -0.49490091],
       [ 1.55236034,  0.90726658],
       [-0.02633256, -0.76271247],
       ...,
       [ 0.09055617, -0.30233549],
       [ 1.63487747,  0.28830632],
       [ 0.288572  , -0.3978091 ]])

In [31]:
# Ajustando el algoritmo con los datos escalados
kmeans6.fit(X_standard)

# Asignando grupos a partir de la variable de entrada
df6['cluster62'] = kmeans6.predict(X_standard)

# Revisando los datos
df6.head()

,Postcode,Average total deductions,Average net tax,cluster61,cluster62
0,2000,2071,27555,3,3
1,2006,3804,28142,1,1
2,2007,1740,15649,0,0
3,2008,3917,53976,1,1
4,2009,3433,32430,1,1


In [32]:
# Instanciando un gráfico de dispersión
scatter_plot62 = alt.Chart(df6).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plot62.encode(x = 'Average net tax',
                      y = 'Average total deductions',
                      color = 'cluster62:N',
                      tooltip = ['Postcode', 'cluster62',
                                 'Average total deductions',
                                 'Average net tax']).properties(width = 495,
                                                                height = 330).interactive()

alt.Chart(...)

### Actividad 5.01: Realizar un análisis de segmentación de clientes en un banco utilizando k-medias
En esta actividad se realiza un análisis de segmentación de clientes. Se utilizan datos del departamento de crédito de un banco internacional que está revisando sus ofertas y desea comprender mejor a sus clientes actuales. Se realiza un análisis de agrupamiento con k-medias para identificar los grupos de clientes similares.

Los datos utilizados son del ***German Credit Dataset*** tomados del UCI Machine Learning Repository.

In [33]:
# Url del archivo
file_urla = 'https://raw.githubusercontent.com/PacktWorkshops/'\
'The-Data-Science-Workshop/master/Chapter05/DataSet/german.data-numeric'

# Generating column names
column_names = [f'X{i}' for i in range(25)]

# Cargando datos
dfa = pd.read_csv(file_urla, header=None, sep= '\\s+', names=column_names)

# Mostrando las primeras filas del dataframe
dfa.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2


In [34]:
# Extrayendo los datos de entrenamiento
Xa = dfa[['X3', 'X9']]

# Instanciando el escalador StandardScaler
standard_scaler = StandardScaler()

# Escalando los datos
Xa_standard = standard_scaler.fit_transform(Xa)

# Mostrando los datos escalados
Xa_standard

array([[-0.73343195,  2.76645648],
       [ 0.96637654, -1.19140394],
       [-0.41471786,  1.18331231],
       ...,
       [-0.87508266,  0.21583532],
       [-0.52095589, -1.10345149],
       [ 0.47059906, -0.75164167]])

In [35]:
# Creando un dataframe y una lista vacia
clustersa = pd.DataFrame()
inertiaa = []

# Añadiendo una columna al dataframe con un clusters de 0 a 14
clustersa['cluster_range'] = range(1, 15)

# Recorriendo cada clúster, ajustandoles un modelo k-medias y
# agregando los valores a la lista 'inertia'
for k in clustersa['cluster_range']:
  kmeans = KMeans(n_clusters = k).fit(Xa_standard)
  inertiaa.append(kmeans.inertia_)

# Agregando la lista 'inertia' al dataframe 'clusters'
clustersa['inertia'] = inertiaa
clustersa

,cluster_range,inertia
0,1,2000.000000
1,2,1297.338101
2,3,700.800711
3,4,559.389521
4,5,508.152875
5,6,371.906085
6,7,318.678613
7,8,299.274164
8,9,251.723993
9,10,232.778813


In [36]:
# Trazando el gráfico Elbow
alt.Chart(clustersa).mark_line().encode(x = 'cluster_range',
                                        y = 'inertia').properties(width = 495,
                                                                  height = 330)

alt.Chart(...)

In [37]:
# Identificando el número óptimo de clústeres y asignando el valor a una variable
clusters_opta = 3

# Entrenando un modelo k-means con esta cantidad de clústeres
kmeansa = KMeans(random_state = 42, n_clusters = clusters_opta)

# Ajustando el modelo con los datos de entrenamiento
kmeansa.fit(Xa_standard)

# Asignación de agrupamiento a partir de la variable de entrada
dfa['Cluster_a'] = kmeansa.predict(Xa_standard)

dfa.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X16,X17,X18,X19,X20,X21,X22,X23,X24,Cluster_a
0,1,6,4,12,5,5,3,4,1,67,...,0,1,0,0,1,0,0,1,1,2
1,2,48,2,60,1,3,2,2,1,22,...,0,1,0,0,1,0,0,1,2,1
2,4,12,4,21,1,4,3,3,1,49,...,0,1,0,0,1,0,1,0,1,2
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,1,1,0
4,1,24,3,49,1,3,3,4,4,53,...,0,1,0,0,0,0,0,1,2,2


In [38]:
# Instanciando un gráfico de dispersión
scatter_plota = alt.Chart(dfa).mark_circle()

# Especificando la visualización del gráfico de dispersión y opciones de interactividad
scatter_plota.encode(x = 'X3',
                     y = 'X9',
                     color = 'Cluster_a:N').properties(width = 495,
                                                       height = 330)

alt.Chart(...)